In [16]:
import os
import time
import cv2
import torch
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import torchvision.models as models
print(os.getcwd())
os.chdir('..')

device = torch.device("cpu")

/home


In [13]:
# open image with cv2 from folder: data/gazebo_data/train1/image0.png
image = cv2.imread(os.path.join("data", "gazebo_data", "train1", "image0.png"))
# convert image to numpy 
image = np.array(image)

print(image)

None


[ WARN:0@159.577] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/gazebo_data/train1/image0.png'): can't open/read file: check file path/integrity


In [14]:
########### MOBILE NET ########### 
model = models.mobilenet_v2(pretrained=True)
model.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

# MobileNetV2 uses a different attribute for the classifier
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Sequential(
torch.nn.Linear(num_ftrs, 512),
torch.nn.BatchNorm1d(512),
torch.nn.ReLU(inplace=True),
torch.nn.Linear(512, 256),
torch.nn.BatchNorm1d(256),
torch.nn.ReLU(inplace=True),
torch.nn.Linear(256, 3)
)

path = os.getcwd() + '/models/' + 'model_005_17-01-2024_15-38-12.pth'
checkpoint = torch.load(path, map_location='cpu')  # Load to CPU
model.load_state_dict(checkpoint)
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: '/home/tommaselli/models/model_005_17-01-2024_15-38-12.pth'

In [ ]:
# Inicie a contagem de tempo antes da inferência
start_time = time.time()

# get the model predictions
predictions = model(image)

# Encerre a contagem de tempo após a inferência
end_time = time.time()

print('Inference time: {:.4f} ms'.format((end_time - start_time)*1000))

In [ ]:
def deprocess(image, label):
    ''' Returns the deprocessed image and label. '''

    if len(label) == 3:
        # we suppose m1 = m2, so we can use the same deprocess
        print('supposing m1 = m2')   
        w1, q1, q2 = label
        w2 = w1
    elif len(label) == 4:
        print('not supposing m1 = m2')        
        w1, w2, q1, q2 = label

    # DEPROCESS THE LABEL
    q1_original = ((q1 + 1) * (187.15 - (-56.06)) / 2) + (-56.06)
    q2_original = ((q2 + 1) * (299.99 - 36.81) / 2) + 36.81
    w1_original = ((w1 + 1) * (0.58 - (-0.58)) / 2) + (-0.58)
    w2_original = ((w2 + 1) * (0.58 - (-0.58)) / 2) + (-0.58)


    # print(f'labels w1={w1}, w2={w2}, q1={q1}, q2={q2}')
    m1, m2, b1, b2 = PreProcess.parametrization(w1_original, w2_original, q1_original, q2_original)

    label = [m1, m2, b1, b2]

    return label

In [ ]:
# convert the predictions to numpy array
predictions = predictions.to('cpu').cpu().detach().numpy()
predictions = PreProcess.deprocess(image=image, label=predictions[0].tolist())


# convert image to cpu 
image = image.to('cpu').cpu().detach().numpy()
# image it is shape (1, 1, 507, 507), we need to remove the first dimension
image = image[0][0]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), frameon=True)

linewidth = 2.5

ax.plot(x, y1p, color='red', label='Predicted', linewidth=linewidth)
ax.plot(x, y2p, color='red', linewidth=linewidth)

# Customize the title
# title_font = {'family': 'Ubuntu', 'color': 'black', 'weight': 'bold', 'size': 18}
# plt.title('Inference Results', fontdict=title_font)

# Add a white rectangle to create a border
border_style = dict(facecolor='none', edgecolor='black', linewidth=2)
ax.add_patch(plt.Rectangle((0, 0), 1, 1, **border_style, transform=ax.transAxes))

# legend
plt.legend(loc='upper right', prop={'size': 9, 'family': 'Ubuntu'})

# show the image
im = ax.imshow(image, cmap='magma', norm=PowerNorm(gamma=16), alpha=0.65)
ax.axis('off')

plt.show()